HW5 -- Dave Foote
Thank you guys so much for letting us redo this, I feel like this one's gonna be a lot better.

In [1]:
%load_ext autoreload
%autoreload 2
import ml_loop as ml
import model_analyzer as ma
import numpy as np

Read Data:

In [2]:
df = ml.read_csv_handledates('data/projects_2012_2013.csv', ['date_posted', 'datefullyfunded'])

Dave - Explore Data Above Here?

Lets calculate some outcomes:

In [3]:
ml.calculate_duration(df, 'date_posted', 'datefullyfunded', 'duration')

In [4]:
ml.compare_tdelta_generate_binary(df, 'duration', '60 days', 'Y')

In [5]:
df.columns

Index(['projectid', 'teacher_acctid', 'schoolid', 'school_ncesid',
       'school_latitude', 'school_longitude', 'school_city', 'school_state',
       'school_metro', 'school_district', 'school_county', 'school_charter',
       'school_magnet', 'teacher_prefix', 'primary_focus_subject',
       'primary_focus_area', 'secondary_focus_subject', 'secondary_focus_area',
       'resource_type', 'poverty_level', 'grade_level',
       'total_price_including_optional_support', 'students_reached',
       'eligible_double_your_impact_match', 'date_posted', 'datefullyfunded',
       'duration', 'Y'],
      dtype='object')

Wanna check and see that 1's represent rows with durations <= 60 days

In [6]:
df[['duration', 'Y']]

,duration,Y
0,18 days,1
1,11 days,1
2,76 days,0
3,55 days,1
4,76 days,0
5,88 days,0
6,22 days,1
7,10 days,1
8,23 days,1
9,31 days,1


Feature Generation

In [7]:
#lets identify the string and float columns per the lab
strcols, fltcols = ml.id_potential_features(df)
print('strings: ', strcols)
print('floats: ', fltcols)

strings:  ['school_metro', 'school_charter', 'school_magnet', 'teacher_prefix', 'primary_focus_area', 'secondary_focus_area', 'resource_type', 'poverty_level', 'grade_level', 'eligible_double_your_impact_match']
floats:  ['school_ncesid', 'school_latitude', 'school_longitude', 'total_price_including_optional_support', 'students_reached']


But we I don't want lat and long or school id as features, because stupid

In [8]:
fltcols.remove('school_latitude')
fltcols.remove('school_longitude')
fltcols.remove('school_ncesid')
print(fltcols)

['total_price_including_optional_support', 'students_reached']


In [9]:
features = ml.strings_to_dummies(df, strcols)

In [10]:
#add the floaty boys
ml.discretize_float_cols(df, features, fltcols, 5)

In [11]:
features.columns

Index(['school_metro_suburban', 'school_metro_urban', 'school_metro_nan',
       'school_charter_t', 'school_charter_nan', 'school_magnet_t',
       'school_magnet_nan', 'teacher_prefix_Mr.', 'teacher_prefix_Mrs.',
       'teacher_prefix_Ms.', 'teacher_prefix_nan',
       'primary_focus_area_Health & Sports',
       'primary_focus_area_History & Civics',
       'primary_focus_area_Literacy & Language',
       'primary_focus_area_Math & Science',
       'primary_focus_area_Music & The Arts',
       'primary_focus_area_Special Needs', 'primary_focus_area_nan',
       'secondary_focus_area_Health & Sports',
       'secondary_focus_area_History & Civics',
       'secondary_focus_area_Literacy & Language',
       'secondary_focus_area_Math & Science',
       'secondary_focus_area_Music & The Arts',
       'secondary_focus_area_Special Needs', 'secondary_focus_area_nan',
       'resource_type_Other', 'resource_type_Supplies',
       'resource_type_Technology', 'resource_type_Trips',
       '

more features, aggregated features

In [12]:
df.columns

Index(['projectid', 'teacher_acctid', 'schoolid', 'school_ncesid',
       'school_latitude', 'school_longitude', 'school_city', 'school_state',
       'school_metro', 'school_district', 'school_county', 'school_charter',
       'school_magnet', 'teacher_prefix', 'primary_focus_subject',
       'primary_focus_area', 'secondary_focus_subject', 'secondary_focus_area',
       'resource_type', 'poverty_level', 'grade_level',
       'total_price_including_optional_support', 'students_reached',
       'eligible_double_your_impact_match', 'date_posted', 'datefullyfunded',
       'duration', 'Y'],
      dtype='object')

In [13]:
features.head()

,school_metro_suburban,school_metro_urban,school_metro_nan,school_charter_t,school_charter_nan,school_magnet_t,school_magnet_nan,teacher_prefix_Mr.,teacher_prefix_Mrs.,teacher_prefix_Ms.,...,poverty_level_moderate poverty,poverty_level_nan,grade_level_Grades 6-8,grade_level_Grades 9-12,grade_level_Grades PreK-2,grade_level_nan,eligible_double_your_impact_match_t,eligible_double_your_impact_match_nan,total_price_including_optional_support,students_reached
0,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,very low,very low
1,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,very low,very low
2,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,very low,very low
3,0,1,0,0,0,1,0,0,0,1,...,0,0,0,0,1,0,0,0,very low,very low
4,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,very low,very low


In [14]:
features['cheap_not_high_impact'] = np.where((features['total_price_including_optional_support'] == 'low') &
                                        (features['students_reached'] == 'low'), 1, 0)

In [15]:
features['cheap_not_high_impact'].sum()

0

In [16]:
features['dollar_per_student'] = df['total_price_including_optional_support'] // df['students_reached']

In [17]:
ml.discretize_float_cols(features, features, ['dollar_per_student'], 5)

In [18]:
features.groupby('dollar_per_student').students_reached.count()

dollar_per_student
very low     124878
low              22
medium           10
high              4
very high         3
Name: students_reached, dtype: int64

In [19]:
df.columns

Index(['projectid', 'teacher_acctid', 'schoolid', 'school_ncesid',
       'school_latitude', 'school_longitude', 'school_city', 'school_state',
       'school_metro', 'school_district', 'school_county', 'school_charter',
       'school_magnet', 'teacher_prefix', 'primary_focus_subject',
       'primary_focus_area', 'secondary_focus_subject', 'secondary_focus_area',
       'resource_type', 'poverty_level', 'grade_level',
       'total_price_including_optional_support', 'students_reached',
       'eligible_double_your_impact_match', 'date_posted', 'datefullyfunded',
       'duration', 'Y'],
      dtype='object')

In [ ]:
features['projects_funded_in_community_type'] = 